In [1]:
!pip install geopandas
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 115.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 120.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 145.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 112.4 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd
import os
import geopandas as gpd
import csv
import rasterio
from rasterio.mask import mask
import numpy as np

In [3]:
with open('processed_EAs.csv', newline='') as f:
    processed_EAs = [line.rstrip('\n') for line in f]

In [4]:
def crop_tif_with_shapefile(tif_dir, shapefile_path, output_dir):
    # Load the shapefile
    gdf = gpd.read_file(shapefile_path)

    # Combine all polygons in the shapefile into a single geometry
    unified_geometry = gdf.unary_union

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for root, dirs, files in os.walk(tif_dir):
        for file in files:
            if file.endswith('.tif'):
                tif_path = os.path.join(root, file)
                with rasterio.open(tif_path) as src:
                    out_image, out_transform = mask(src, [unified_geometry], crop=True, nodata=np.nan)
                    out_meta = src.meta

                    # Update the metadata to reflect the number of layers, if necessary
                    out_meta.update({"driver": "GTiff",
                                     "height": out_image.shape[1],
                                     "width": out_image.shape[2],
                                     "transform": out_transform,
                                     "nodata": np.nan})

                    output_path = os.path.join(output_dir, file)
                    with rasterio.open(output_path, "w", **out_meta) as dest:
                        dest.write(out_image)

# Example usage
tif_dirs = ['MODIS_bands', 'NDBI']
shapefile_path = 'GAMA_boundary_satellite/GAMA_boundary_satellite.shp'

for tif_dir in tif_dirs:
    output_dir = f'cropped_{tif_dir}'  # Define your output directory
    crop_tif_with_shapefile(tif_dir, shapefile_path, output_dir)
